<a href="https://colab.research.google.com/github/DanaJian/Machine-learning-technologies/blob/main/Analysis_and_prediction_of_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter

import plotly.graph_objects as go
import plotly.subplots as sp

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sma

from scipy.stats import boxcox

import itertools

import warnings
warnings.filterwarnings('ignore')

1. ЗАГРУЗКА ДАННЫХ

In [ ]:
data = pd.read_csv('/content/Time_Series.csv', index_col=0)
data

,came
date,
2016-01-10,1345
2016-01-17,2066
2016-01-24,1979
2016-01-31,1909
2016-02-07,1575
...,...
2019-03-03,1483
2019-03-10,1172
2019-03-17,1435


In [ ]:
data.dtypes

came    int64
dtype: object

2. ПРОВЕДЕНИЕ ТЕСТА ДИКИ-ФУЛЛЕРА

In [ ]:
result = adfuller(data['came'])

if result[1] > 0.05:
    print("p-value:", result[1])
    print("Нулевая гипотеза не отвергается: ряд нестационарен")
else:
    print("p-value:", result[1])
    print("Альтернативная гипотеза не отвергнута: ряд стационарен")

p-value: 0.6320331972965831
Нулевая гипотеза не отвергается: ряд нестационарен


3. РЕАЛИЗАЦИЯ ДЕКОМПОЗИЦИИ ВРЕМЕННОГО РЯДА

In [ ]:
period = 12
decomp = sma.tsa.seasonal_decompose(data, period=period)

fig = sp.make_subplots(rows=4, cols=1)
fig.add_trace(go.Scatter(x=data.index, y=data['came'], name='Оригинальный ряд'), row=1, col=1)
fig.update_yaxes(title_text="Оригинальный ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=decomp.trend, name='Тренд'), row=2, col=1)
fig.update_yaxes(title_text="Тренд", row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=decomp.seasonal, name='Сезонность'), row=3, col=1)
fig.update_yaxes(title_text="Сезонность", row=3, col=1)
fig.add_trace(go.Scatter(x=data.index, y=decomp.resid, name='Шум'), row=4, col=1)
fig.update_yaxes(title_text="Шум", row=4, col=1)
fig.update_traces(showlegend=False)
fig.update_layout(title='Декомпозиция временного ряда', height=1000)
fig.show()

4. ПРЕОБРАЗОВАНИЕ БОКСА-КОКСА

In [ ]:
lamda = boxcox(data.came)[1]
print("Оптимальное значение lamda для преобразования: ", lamda)

data['boxcox_came'] = boxcox(data.came, lamda)
fig = sp.make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=data.index, y=data.came), row=1, col=1)
fig.update_yaxes(title_text="Временной ряд", row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data.boxcox_came), row=2, col=1)
fig.update_yaxes(title_text="Преобразованнный ряд", row=2, col=1)
fig.update_layout(title='Преобразование Бокса-Кокса', height=1000)
fig.show()

Оптимальное значение lamda для преобразования:  0.11738125874670109


In [ ]:
result = adfuller(data.boxcox_came)

if result[1] > 0.05:
    print("p-value:", result[1])
    print("Нулевая гипотеза не отвергается: ряд нестационарен")
else:
    print("p-value:", result[1])
    print("Альтернативная гипотеза не отвергнута: ряд стационарен")

p-value: 0.5348617481593065
Нулевая гипотеза не отвергается: ряд нестационарен


5. ОБУЧЕНИЕ МОДЕЛИ ARIMA И ПРЕДСКАЗАНИЕ НА 12 ОТСЧЕТОВ ВПЕРЕД

In [ ]:
# Подбор p, d, q параметров для модели ARIMA
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))] 
print('Примеры комбинаций параметров для модели SARIMA:')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

Примеры комбинаций параметров для модели SARIMA:
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [ ]:
# Поиск по сетке
min_aic = float("inf")
min_params = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sma.tsa.statespace.SARIMAX(data.came,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            aic = results.aic
            if aic < min_aic:
                min_aic = aic
                min_params = (param, param_seasonal)
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, aic))
        except:
            continue

print(f"Best model: ARIMA{min_params[0]}x{min_params[1]}12 with AIC = {min_aic:.2f}")

ARIMA(0, 0, 0)x(0, 0, 0, 12)12 - AIC:2992.0086354327686
ARIMA(0, 0, 0)x(0, 0, 1, 12)12 - AIC:2602.311625088495
ARIMA(0, 0, 0)x(0, 1, 0, 12)12 - AIC:2176.5909696792023
ARIMA(0, 0, 0)x(0, 1, 1, 12)12 - AIC:1991.4255099649195
ARIMA(0, 0, 0)x(1, 0, 0, 12)12 - AIC:2187.5578554772355
ARIMA(0, 0, 0)x(1, 0, 1, 12)12 - AIC:2160.8370974586614
ARIMA(0, 0, 0)x(1, 1, 0, 12)12 - AIC:2003.7907494898027
ARIMA(0, 0, 0)x(1, 1, 1, 12)12 - AIC:1977.6015154572347
ARIMA(0, 0, 1)x(0, 0, 0, 12)12 - AIC:2768.4189887325965
ARIMA(0, 0, 1)x(0, 0, 1, 12)12 - AIC:2451.5809747521103
ARIMA(0, 0, 1)x(0, 1, 0, 12)12 - AIC:2110.149115760545
ARIMA(0, 0, 1)x(0, 1, 1, 12)12 - AIC:1915.6118222998177
ARIMA(0, 0, 1)x(1, 0, 0, 12)12 - AIC:2141.5348027607497
ARIMA(0, 0, 1)x(1, 0, 1, 12)12 - AIC:2087.9151036089165
ARIMA(0, 0, 1)x(1, 1, 0, 12)12 - AIC:1942.080482127762
ARIMA(0, 0, 1)x(1, 1, 1, 12)12 - AIC:1908.229858364214
ARIMA(0, 1, 0)x(0, 0, 0, 12)12 - AIC:2153.098414471574
ARIMA(0, 1, 0)x(0, 0, 1, 12)12 - AIC:1997.54451510286

In [ ]:
model = ARIMA(data.came, order=(0, 1, 1))
results = model.fit()
forecast = results.forecast(steps=12)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data.came, name='Временной ряд'))
fig.add_trace(go.Scatter(x=forecast.index, y=forecast.values, name="Спрогнозированные данные", mode='lines', line=dict(color='red')))
fig.update_layout(title='Прогноз на 12 отсчетов вперед')
fig.show()

6. ВЫЧИСЛЕНИЕ МЕТРИК MAPE, SMAPE И MAE

In [ ]:
data = data.drop('boxcox_came', axis=1)
train = data.loc['2016-01-10':'2018-03-04']
test = data.loc['2018-03-11':]

In [ ]:
model = ARIMA(train.came, order=(0, 1, 1))
results = model.fit()
forecast = results.forecast(steps=len(test))

In [ ]:
metrics = {
    'mae': np.mean(np.abs(forecast.values - test.came)),
    'mape': np.mean(np.abs((test.came - forecast.values) / test.came)) * 100,
    'smape': np.mean(2 * np.abs(forecast.values - test.came) / (np.abs(test.came) + np.abs(forecast.values))) * 100
}

metrics_df = pd.DataFrame(metrics, index=['Значение'])
metrics_df

,mae,mape,smape
Значение,131.120606,9.687376,9.085987
